In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
! pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 994.0/994.0 kB 24.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 76.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 59.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 88.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstall

In [3]:
!unzip /content/drive/MyDrive/Capstone/Yolo+deepsort/deep_sort.zip

Archive:  /content/drive/MyDrive/Capstone/Yolo+deepsort/deep_sort.zip
   creating: deep_sort/
  inflating: deep_sort/.gitignore    
   creating: deep_sort/application_util/
  inflating: deep_sort/application_util/image_viewer.py  
  inflating: deep_sort/application_util/preprocessing.py  
  inflating: deep_sort/application_util/visualization.py  
 extracting: deep_sort/application_util/__init__.py  
   creating: deep_sort/deep_sort/
  inflating: deep_sort/deep_sort/detection.py  
  inflating: deep_sort/deep_sort/iou_matching.py  
  inflating: deep_sort/deep_sort/kalman_filter.py  
  inflating: deep_sort/deep_sort/linear_assignment.py  
  inflating: deep_sort/deep_sort/nn_matching.py  
  inflating: deep_sort/deep_sort/track.py  
  inflating: deep_sort/deep_sort/tracker.py  
 extracting: deep_sort/deep_sort/__init__.py  
   creating: deep_sort/deep_sort/__pycache__/
  inflating: deep_sort/deep_sort/__pycache__/detection.cpython-312.pyc  
  inflating: deep_sort/deep_sort/__pycache__/iou_m

In [4]:
###객체추적_v3_deepsort2

import cv2
import numpy as np
import pandas as pd
from ultralytics import YOLO

# DeepSORT 관련 모듈
from deep_sort.deep_sort.tracker import Tracker as DeepSortTracker
from deep_sort.tools import generate_detections as gdet
from deep_sort.deep_sort import nn_matching
from deep_sort.deep_sort.detection import Detection
from collections import Counter

print("Imports done.")

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


Instructions for updating:
non-resource variables are not supported in the long term


Imports done.


In [5]:
mars_small128 = "/content/drive/MyDrive/Capstone/Yolo+deepsort/mars-small128.pb"

In [6]:
### Deepsort 연결을 위한 class 2개 생성성
class Track:
    def __init__(self, track_id, bbox):
        self.track_id = track_id
        self.bbox = bbox


class DeepSortWrapper:
    def __init__(self, model_filename= mars_small128, max_cosine_distance=0.4, nn_budget=None):
        metric = nn_matching.NearestNeighborDistanceMetric("cosine", max_cosine_distance, nn_budget)
        self.tracker = DeepSortTracker(metric)
        self.encoder = gdet.create_box_encoder(model_filename, batch_size=1)
        self.tracks = []

    def update(self, frame, detections):

        # Step 1: If no detections, run a predict-update cycle with an empty list.
        if len(detections) == 0:
            self.tracker.predict()
            self.tracker.update([])
            self._update_tracks()
            return

        # Step 2: Convert [x1, y1, x2, y2] to [x, y, w, h] for Deep SORT
        bboxes = np.array([d[:4] for d in detections])
        scores = [d[4] for d in detections]
        bboxes[:, 2:] = bboxes[:, 2:] - bboxes[:, :2]

        # Step 3: Generate appearance features for each bounding box
        features = self.encoder(frame, bboxes)

        # Step 4: Wrap everything in Deep SORTs Detection objects
        dets = []
        for bbox_id, bbox in enumerate(bboxes):
            dets.append(Detection(bbox, scores[bbox_id], features[bbox_id]))

        # Step 5: Predict and update the tracker
        self.tracker.predict()
        self.tracker.update(dets)
        self._update_tracks()

    def _update_tracks(self):
        active_tracks = []
        for track in self.tracker.tracks:
            if not track.is_confirmed() or track.time_since_update > 1:
                continue
            bbox = track.to_tlbr()  # returns [x1, y1, x2, y2]
            track_id = track.track_id
            active_tracks.append(Track(track_id, bbox))

        self.tracks = active_tracks

In [7]:
### tensorflow-gpu 연결에는 문제가 없으나 deepsort의 gpu이용에 문제.
import tensorflow as tf
tf.config.set_visible_devices([], 'GPU')  # DeepSORT가 GPU를 사용하지 않도록 설정


In [ ]:
# import math

# # YOLO 모델 로드
# model_weight_path = "/content/drive/MyDrive/Capstone/best_motorcycle_detector_NIGHT8.pt"
# model = YOLO(model_weight_path)
# model.to('cuda')  # GPU 사용

# # DeepSORT 초기화
# ### DeepSORTWrapper와 Mars
# tracker = DeepSortWrapper(model_filename='/content/mars-small128.pb', max_cosine_distance=0.2, nn_budget=None)
# print("YOLO model and Deep SORT wrapper initialized.")

# # 전역 변수 정의
# log_data = []
# interval_data = []
# detected_classes = []
# object_data = {}
# object_directions = {}
# pair_code_map = {}  # Pair Code 매핑
# pair_code_counter = 1  # Pair Code 생성용
# object_class_counts = {}

# frame_counter = 0
# pixel_to_meter_ratio = None

# # 영역 정의 (YOLO 형식)
# right_turn_zone_yolo = (0.399740, 0.538426, 0.067187, 0.152778)
# straight_zone_yolo = (0.810417, 0.805093, 0.141667, 0.171296)
# interval_zone_yolo = (0.467187, 0.759722, 0.837500, 0.515741)
# finish_zone_yolo = (0.044271, 0.858333, 0.081250, 0.211111)
# actual_interval_width = 47.0  # 실제 평가 거리 (미터)

# # 고유 ID 생성 및 유지
# # def get_unique_id(track_id):
# #     if track_id not in unique_ids:
# #         unique_ids[track_id] = str(uuid.uuid4())[:8]
# #     return unique_ids[track_id]

# # 픽셀-to-미터 변환 비율 계산
# def calculate_pixel_to_meter_ratio(frame_width):
#     interval_width_pixels = interval_zone_yolo[2] * frame_width
#     if interval_width_pixels < 1:
#         print("Error: interval_width_pixels is 0, check interval_zone_yolo values.")
#         return 1.0  # 혹은 기본값 설정
#     pixel_to_meter_ratio = actual_interval_width / interval_width_pixels

#     if pixel_to_meter_ratio < 0.01:
#         print(f"[DEBUG] pixel_to_meter_ratio 값이 너무 작아 기본값(0.1)로 설정합니다.")
#         pixel_to_meter_ratio = 0.1

#     print(f"[DEBUG] Pixel-to-Meter Ratio : {pixel_to_meter_ratio}")
#     return pixel_to_meter_ratio

# # YOLO 형식의 좌표를 픽셀 좌표로 변환
# def yolo_to_pixel(label, img_width, img_height, max_size=2000):
#     # YOLO format: (x_center, y_center, width, height)
#     x_center, y_center, width, height = label

#     # 상대 좌표를 픽셀 좌표로 변환
#     x1 = (x_center - width / 2) * img_width
#     x2 = (x_center + width / 2) * img_width
#     y1 = (y_center - height / 2) * img_height
#     y2 = (y_center + height / 2) * img_height

#     # 바운딩 박스 크기가 너무 커지지 않도록 제한
#     width_pixels = max(0, min(max_size, x2 - x1))
#     height_pixels = max(0, min(max_size, y2 - y1))

#     # 좌표 범위 보정
#     x1 = max(0, min(x1, img_width - width_pixels))
#     x2 = x1 + width_pixels
#     y1 = max(0, min(y1, img_height - height_pixels))
#     y2 = y1 + height_pixels

#     # 디버깅 출력
#     print(f"YOLO Output (x_center, y_center, width, height): {label}")
#     print(f"Converted Pixel BBox: ({x1}, {y1}, {x2}, {y2})")

#     return (int(x1), int(y1), int(x2), int(y2))

# # BBox가 특정 Zone과 교차 또는 포함되는지 확인
# def is_bbox_in_zone(bbox, zone):
#     (x1, y1, x2, y2) = bbox
#     (z_x1, z_y1, z_x2, z_y2) = zone
#     return not (x2 < z_x1 or x1 > z_x2 or y2 < z_y1 or y1 > z_y2)

# # BBox를 기준으로 차량의 방향(우회전/직진)을 분류
# def classify_direction(bbox, frame_width, frame_height):
#     right_turn_zone = yolo_to_pixel(right_turn_zone_yolo, frame_width, frame_height)
#     straight_zone = yolo_to_pixel(straight_zone_yolo, frame_width, frame_height)

#     if is_bbox_in_zone(bbox, right_turn_zone):
#         return "우회전"
#     elif is_bbox_in_zone(bbox, straight_zone):
#         return "직진"
#     return "미정"

# def calculate_angle(dx, dy):
#     if dx == 0:
#         return 90 if dy > 0 else -90  # 위쪽 이동(90도), 아래쪽 이동(-90도)
#     return math.degrees(math.atan2(dy, dx))
# # import math

# # def calculate_angle(dx, dy):
# #     """
# #     이동 벡터 (dx, dy)를 이용해 각도를 계산하는 함수.
# #     :param dx: X축 이동 거리
# #     :param dy: Y축 이동 거리
# #     :return: 이동 각도 (degree)
# #     """
# #     angle = math.degrees(math.atan2(dy, dx))  # atan2는 (-180 ~ 180) 범위 반환
# #     return angle



# def determine_movement_direction(track_id, current_position):
#     """
#     객체의 이동 벡터를 계산하고, 실제 이동 방향을 분석하여 반환하는 함수.
#     - 수평(왼쪽) 이동 → "직진"
#     - 아래쪽 이동 → "우회전"
#     - 그 외 → "미정"

#     :param track_id: 객체의 고유 추적 ID
#     :param current_position: 현재 프레임에서의 객체 중심점 (x, y)
#     :return: 이동 방향 ("직진", "우회전", "미정")
#     """
#     if track_id in object_data:
#         prev_position = object_data[track_id]["position"]

#         # 이동 벡터 계산
#         dx = current_position[0] - prev_position[0]  # X축 이동 거리
#         dy = current_position[1] - prev_position[1]  # Y축 이동 거리

#         # 이동 방향 각도 계산
#        # angle = calculate_angle(dx, dy)

#         # ✅ 이동 방향 판별
#         if dx < 0:  # 왼쪽 방향 (거의 수평 이동)
#             direction = "직진"
#         elif dy > 0:  # 아래쪽 이동 → 우회전 가능
#             direction = "우회전"
#         else:
#             direction = "미정"  # 방향을 알 수 없는 경우

#         # 객체 데이터 업데이트 (현재 위치를 새로운 기준점으로 저장)
#         object_data[track_id] = {"position": current_position, "direction": direction}

#         return direction
#     else:
#         # 처음 감지된 객체 → "미정" 상태로 설정
#         object_data[track_id] = {"position": current_position, "direction": "미정"}
#         return "미정"

# # 이동 거리, 속도, 가속도 계산
# import numpy as np


# def calculate_motion(object_id, current_position, fps, pixel_to_meter_ratio):
#     if object_id in object_data:
#         prev_data = object_data[object_id]
#         prev_pos = prev_data.get("position",current_position)
#         prev_speed = prev_data.get("speed",0.0)

#         # 이전 위치가 현재 위치와 동일하면 거리 계산 제외
#         #if np.array_equal(prev_pos, current_position):
#         #    print(f"[DEBUG] Object {object_id} did not move.")
#         #    return 0.0, prev_speed, 0.0


#         # 거리 및 속도 계산
#         distance_pixels = np.linalg.norm(np.array(current_position) - np.array(prev_pos))
#         distance_meters = distance_pixels * pixel_to_meter_ratio
#         speed = distance_meters * fps
#         acceleration = (speed - prev_speed) * fps

#         # 가속도가 비정상적으로 클 경우 제한
#         if acceleration > 100:
#             print(f"[DEBUG] Object {object_id} - Acceleration 너무 큼: {acceleration:.2f}m/s² → 100으로 제한")
#             acceleration = 100
#                 # 이동이 감지되지 않으면 로그를 남기지 않음

#         if distance_meters < 0.01:
#             return 0.0, prev_speed, 0.0  # 🔹 작은 이동은 무시

#         # 디버깅 정보 출력
#         print(f"[DEBUG] Object {object_id} - Distance: {distance_meters:.2f}m, Speed: {speed:.2f}m/s, Accel: {acceleration:.2f}m/s²")

#         # 객체 정보 업데이트
#         object_data[object_id] = {"position": current_position, "speed": speed}
#         return distance_meters, speed, acceleration

#     else:
#         # 객체가 처음 등장했을 때 초기화
#         object_data[object_id] = {"position": current_position, "speed": 0.0}
#         print(f"[DEBUG] Object {object_id} initialized.")
#         return 0.0, 0.0, 0.0


# # 객체 클래스 변경 함수
# def update_object_class(track_id, new_class):
#     track_id = str(track_id)

#     if track_id not in object_class_counts:
#         object_class_counts[track_id] = Counter()

#     # 새로운 클래스를 카운트
#     object_class_counts[track_id][new_class] += 1

#     # 가장 빈도가 높은 클래스를 가져옴
#     most_common_class = object_class_counts[track_id].most_common(1)[0][0]
#     class_label = "car" if most_common_class == 2 else "motorcycle"

#     # 객체의 클래스가 결정되면 모든 프레임에서 해당 클래스를 업데이트
#     for obj in log_data:
#         if obj["Object ID"] == track_id:
#             obj["Class"] = class_label

#     # interval_data에도 반영 (추가된 부분)
#     for interval in interval_data:
#         if interval["Object 1 ID"] == track_id:
#             interval["Object 1 Class"] = class_label
#         if interval["Object 2 ID"] == track_id:
#             interval["Object 2 Class"] = class_label
#     return most_common_class

# # Interval 데이터 생성
# def update_interval_data(frame_num, log_data, pixel_to_meter_ratio):
#     global interval_data, pair_code_map, pair_code_counter

#     right_turn_vehicles = [obj for obj in log_data if obj["Direction"] == "우회전"]
#     straight_vehicles = [obj for obj in log_data if obj["Direction"] == "직진" or obj["Direction"]=="미정"]

#     for obj1 in right_turn_vehicles:
#         for obj2 in straight_vehicles:
#             if not obj1.get("Object ID") or not obj2.get("Object ID"):
#                 continue

#             # 중심점 거리 계산
#             center1 = ((obj1["BBox"][0] + obj1["BBox"][2]) / 2, (obj1["BBox"][1] + obj1["BBox"][3]) / 2)
#             center2 = ((obj2["BBox"][0] + obj2["BBox"][2]) / 2, (obj2["BBox"][1] + obj2["BBox"][3]) / 2)
#             lag_spacing_pixels = np.linalg.norm(np.array(center1) - np.array(center2))
#             lag_spacing_meters = lag_spacing_pixels * pixel_to_meter_ratio

#             # 고유 Pair Code 생성 및 관리
#             pair_key = (obj1["Object ID"], obj2["Object ID"])
#             if pair_key not in pair_code_map:
#                 pair_code_map[pair_key] = pair_code_counter
#                 pair_code_counter += 1

#             interval_data.append({
#                 "Frame": frame_num,
#                 "Object 1 ID": obj1["Object ID"],
#                 "Object 2 ID": obj2["Object ID"],
#                 "Pair Code": pair_code_map[pair_key],
#                 "Lag Spacing (m)": lag_spacing_meters,
#                 "Object 1 Speed (m/s)": obj1["Speed (m/s)"],
#                 "Object 2 Speed (m/s)": obj2["Speed (m/s)"],
#                 "Object 1 Class": obj1["Class"],
#                 "Object 2 Class": obj2["Class"]
#             })

# # YOLO 결과를 DeepSORT 입력으로 변환
# def yolo_to_deepsort(yolo_results, target_classes):
#     detections = []
#     classes = []
#     for det in yolo_results[0].boxes:
#         x1, y1, x2, y2 = map(float, det.xyxy[0].cpu().numpy())
#         confidence = float(det.conf.cpu().numpy().item())
#         class_id = int(det.cls.cpu().numpy())
#         if class_id in target_classes:
#             detections.append([(x1, y1, x2, y2), confidence])
#             classes.append(class_id)

#             # 디버깅 로그 추가: YOLO에서 DeepSORT로 전달되는 바운딩 박스 및 신뢰도 출력
#             print(f"YOLO to DeepSORT - BBox: ({x1}, {y1}, {x2}, {y2}), Confidence: {confidence}, Class ID: {class_id}")

#     return detections, classes

# # 객체 클래스별 색상 반환 함수
# def get_class_color(class_id):
#     if class_id == 2:  # car
#         return (0, 255, 0)  # 초록 (BGR)
#     else:
#         return (255, 0, 0)  # 파랑 (BGR)

# # 수정된 메인 루프

# if __name__ == "__main__":
#     target_classes = [2, 3]  # car(2), motorcycle(3)

#     video_path = "/content/drive/MyDrive/Capstone/11.15 1800-2000/11.15 1800-1830 - 필터링.mp4"
#     cap = cv2.VideoCapture(video_path)
#     if not cap.isOpened():
#       print(f"Error: Unable to open video file {video_path}")

#     fps = int(cap.get(cv2.CAP_PROP_FPS))

#     if fps <= 0 or fps > 60:
#       print(f"[DEBUG] FPS 값이 비정상적({fps}), 기본값(30)으로 설정합니다.")
#       fps = 30

#     print(f"[DEBUG] FPS: {fps}")

#     frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
#     frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
#     if frame_width == 0 or frame_height == 0:
#       print("Error: Invalid video frame dimensions")

#     pixel_to_meter_ratio = calculate_pixel_to_meter_ratio(frame_width)

#     output_directory = "/content/drive/MyDrive/Capstone/Results/11.15 1800-2000/Log video"
#     import os
#     output_video_path = os.path.join(output_directory, '11.15 1800-1830_processed_videoA.avi')
#     video_writer = cv2.VideoWriter(output_video_path, cv2.VideoWriter_fourcc(*'MJPG'), fps, (frame_width, frame_height))

#     while cap.isOpened():
#         ret, frame = cap.read()
#         if not ret:
#             break

#         frame_counter += 1

#         # YOLO 객체 검출
#         results = model(frame, conf=0.3)

#         valid_classes = {2:"car",3:"motorcycle", 5:"bus", 7:"truck"}
#         class_counts = {"car":0,"motorcycle":0, "bus":0,"truck:0"}

#         # YOLO 결과를 DeepSORT로 변환
#         detected_classes.clear()
#         all_detections = []

#         for r in results:
#             for box in r.boxes.data.tolist():
#               x1, y1, x2, y2, score, class_id = box
#               if score >= 0.5:
#                 class_name = valid_classes[int(class_id)]
#                 class_counts[class_name] = class_counts.get(class_name, 0) + 1
#                 all_detections.append([int(x1),int(y1),int(x2),int(y2),float(score)])
#                 detected_classes.append(int(class_id))
#         # Ensure detected_classes and tracker.tracks have the same length
#         if len(all_detections) != len(detected_classes):
#             print(f"Warning: Mismatch between detections ({len(all_detections)}) and detected classes ({len(detected_classes)})")

#         # DeepSORTWrapper로 추적
#         tracker.update(frame, all_detections)
#         if not detected_classes:
#             print("Warning: No detected classes were found, skipping classification.")

#         current_log_data = []
#         for track, detected_class_id in zip(tracker.tracks,detected_classes):
#             track_id = track.track_id
#             x1, y1, x2, y2 = track.bbox

#             # DeepSORT에서 제공하는 track_id를 사용하여 class_id를 가져옴
#             #class_id = update_object_class(track_id, 2)  # 기본값 2(car)로 설정

#             # 디버깅: 클래스 ID와 색상 확인
#             print(f"Track ID: {track_id}, Updated Class ID: {class_id}")

#             updated_class_id = update_object_class(track_id, detected_class_id)

#             color = get_class_color(updated_class_id)  # 🔹 color 변수 정의 추가

#             cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), color, 2)

#             if f"ID {track_id}" not in log_data:
#                 label = "car" if updated_class_id == 2 else "motorcycle"
#                 direction = object_directions.get(track_id, "미정")

#                 cv2.putText(frame, f"{label} ID {track_id} {direction}",
#                             (int(x1), int(y1) - 10),
#                             cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)


#             # 바운딩 박스 x좌표가 음수인 경우, 추적 중단
#             if x1 < 0 or y1 < 0 or x2 < 0 or y2 < 0:
#                 continue

#             current_position = ((x1 + x2) // 2, (y1 + y2) // 2)
#             print(f"[DEBUG] Frame {frame_counter} - Object {track_id} Previous Position: {object_data.get(track_id, {}).get('position', 'N/A')} -> Current Position: {current_position}")

#             #unique_id = get_unique_id(track_id)



#             # 이동 거리, 속도, 가속도 계산
#             distance, speed, acceleration = calculate_motion(track_id, current_position, fps, pixel_to_meter_ratio)

#             # 이동 벡터를 기반으로 실제 이동 방향 결정
#             direction = determine_movement_direction(track_id, current_position)

#             bbox = (int(x1), int(y1), int(x2), int(y2))

#             # 방향 분류
#             direction = object_directions.get(track_id, "미정")
#             if direction == "미정":
#                 direction = classify_direction(bbox, frame_width, frame_height)
#                 object_directions[track_id] = direction

#             # 로그 데이터 기록
#             bbox = (int(x1), int(y1), int(x2), int(y2))

#             #if not any(obj["Object ID"] == track_id for obj in log_data):
#             if is_bbox_in_zone(bbox, yolo_to_pixel(interval_zone_yolo, frame_width, frame_height)):
#                 direction = classify_direction(bbox, frame_width, frame_height)

#                 obj_data = {
#                     "Frame": frame_counter,
#                     "Object ID": track_id,
#                     "Class": "car" if updated_class_id == 2 else "motorcycle",
#                     "Distance (m)": distance,
#                     "Speed (m/s)": speed,
#                     "Acceleration (m/s^2)": acceleration,
#                     "Direction": direction,
#                     "BBox": (x1, y1, x2, y2)
#                 }
#             log_data.append(obj_data)
#             current_log_data.append(obj_data)

#             # 화면 표시
#             label = ""
#             if updated_class_id == 2:
#                 label = "car"
#             elif updated_class_id == 3:
#                 label = "motorcycle"

#             color = get_class_color(updated_class_id)
# #            cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
#             cv2.putText(frame, f"{label} ID {track_id} {direction}", (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5,color, 2)

#         # Interval 데이터 업데이트
#         update_interval_data(frame_counter, current_log_data, pixel_to_meter_ratio)

#         if not video_writer.isOpened():
#             print("Error: VideoWriter failed to open.")

#         if frame is None:
#             print(f"Error: Empty frame at {frame_counter}")




#         # 비디오 저장
#         video_writer.write(frame)
#         # 객체 ID 및 바운딩 박스 정보 로그 출력
#         #print(f"Object ID: {unique_id}, BBox: ({x1}, {y1}, {x2}, {y2}), Direction: {direction}")


#     # 데이터 저장
#     log_data_path = "/content/drive/MyDrive/Capstone/Results/11.15 1800-2000/Log data/1800-1830_logA.csv"
#     interval_data_path = "/content/drive/MyDrive/Capstone/Results/11.15 1800-2000/Interval data/1800-1830_intervalA.csv"
#     pd.DataFrame(log_data).to_csv(log_data_path, index=False)
#     pd.DataFrame(interval_data).to_csv(interval_data_path, index=False)


#     print(f"Log data saved to: {log_data_path}")
#     print(f"Interval data saved to: {interval_data_path}")

#     cap.release()
#     video_writer.release()

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
Converted Pixel BBox: (703.00128, 498.99996, 832.00032, 664.0002)
YOLO Output (x_center, y_center, width, height): (0.810417, 0.805093, 0.141667, 0.171296)
Converted Pixel BBox: (1420.00032, 777.0005999999998, 1692.00096, 962.00028)
Track ID: 189, Updated Class ID: 2.0
[DEBUG] Frame 2282 - Object 189 Previous Position: (1222.0, 731.0) -> Current Position: (1218.0, 730.0)
[DEBUG] Object 189 - Acceleration 너무 큼: 108.46m/s² → 100으로 제한
[DEBUG] Object 189 - Distance: 0.12m, Speed: 3.62m/s, Accel: 100.00m/s²
YOLO Output (x_center, y_center, width, height): (0.467187, 0.759722, 0.8375, 0.515741)
Converted Pixel BBox: (92.99904000000001, 522.99972, 1700.99904, 1080.0)
YOLO Output (x_center, y_center, width, height): (0.39974, 0.538426, 0.067187, 0.152778)
Converted Pixel BBox: (703.00128, 498.99996, 832.00032, 664.0002)
YOLO Output (x_center, y_center, width, height): (0.810417, 0.805093, 0.141667, 0.171296)
Converted Pixel BBox: (1420.00032, 777.00059999999

In [ ]:
### x축 거리 차 +density 추가


import math
import cv2
import os
import numpy as np
import pandas as pd
from collections import Counter

# YOLO 모델 로드
model_weight_path = "/content/drive/MyDrive/Capstone/best_motorcycle_detector_DAY&NIGHT8.pt"
model = YOLO(model_weight_path)
model.to('cuda')  # GPU 사용

# DeepSORT 초기화
### DeepSORTWrapper와 Mars
tracker = DeepSortWrapper(model_filename= mars_small128, max_cosine_distance=0.2, nn_budget=None)
print("YOLO model and Deep SORT wrapper initialized.")

# 전역 변수 정의
log_data = []
interval_data = []
detected_classes = []
object_data = {}
object_directions = {}
pair_code_map = {}  # Pair Code 매핑
pair_code_counter = 1  # Pair Code 생성용
object_class_counts = {}

frame_counter = 0
pixel_to_meter_ratio = None
density_per_frame = {}

# 영역 정의 (YOLO 형식)
right_turn_zone_yolo = (0.399740, 0.538426, 0.067187, 0.152778)
straight_zone_yolo = (0.810417, 0.805093, 0.141667, 0.171296)
interval_zone_yolo = (0.399219, 0.706481, 0.789062, 0.431481)
finish_zone_yolo = (0.044271, 0.858333, 0.081250, 0.211111)
actual_interval_width = 47.0  # 실제 평가 거리 (미터)

# 고유 ID 생성 및 유지
# def get_unique_id(track_id):
#     if track_id not in unique_ids:
#         unique_ids[track_id] = str(uuid.uuid4())[:8]
#     return unique_ids[track_id]

# 픽셀-to-미터 변환 비율 계산
def calculate_pixel_to_meter_ratio(frame_width):
    interval_width_pixels = interval_zone_yolo[2] * frame_width
    if interval_width_pixels < 1:
        print("Error: interval_width_pixels is 0, check interval_zone_yolo values.")
        return 1.0  # 혹은 기본값 설정
    pixel_to_meter_ratio = actual_interval_width / interval_width_pixels

    if pixel_to_meter_ratio < 0.01:
        print(f"[DEBUG] pixel_to_meter_ratio 값이 너무 작아 기본값(0.1)로 설정합니다.")
        pixel_to_meter_ratio = 0.1

    # print(f"[DEBUG] Pixel-to-Meter Ratio : {pixel_to_meter_ratio}")
    return pixel_to_meter_ratio

# YOLO 형식의 좌표를 픽셀 좌표로 변환
def yolo_to_pixel(label, img_width, img_height, max_size=2000):
    # YOLO format: (x_center, y_center, width, height)
    x_center, y_center, width, height = label

    # 상대 좌표를 픽셀 좌표로 변환
    x1 = (x_center - width / 2) * img_width
    x2 = (x_center + width / 2) * img_width
    y1 = (y_center - height / 2) * img_height
    y2 = (y_center + height / 2) * img_height

    # 바운딩 박스 크기가 너무 커지지 않도록 제한
    width_pixels = max(0, min(max_size, x2 - x1))
    height_pixels = max(0, min(max_size, y2 - y1))

    # 좌표 범위 보정
    x1 = max(0, min(x1, img_width - width_pixels))
    x2 = x1 + width_pixels
    y1 = max(0, min(y1, img_height - height_pixels))
    y2 = y1 + height_pixels

    # 디버깅 출력
    #print(f"YOLO Output (x_center, y_center, width, height): {label}")
    #print(f"Converted Pixel BBox: ({x1}, {y1}, {x2}, {y2})")

    return (int(x1), int(y1), int(x2), int(y2))

# BBox가 특정 Zone과 교차 또는 포함되는지 확인
def is_bbox_in_zone(bbox, zone):
    (x1, y1, x2, y2) = bbox
    (z_x1, z_y1, z_x2, z_y2) = zone
    return not (x2 < z_x1 or x1 > z_x2 or y2 < z_y1 or y1 > z_y2)

# BBox를 기준으로 차량의 방향(우회전/직진)을 분류
def classify_direction(bbox, frame_width, frame_height):
    right_turn_zone = yolo_to_pixel(right_turn_zone_yolo, frame_width, frame_height)
    straight_zone = yolo_to_pixel(straight_zone_yolo, frame_width, frame_height)

    if is_bbox_in_zone(bbox, right_turn_zone):
        return "우회전"
    elif is_bbox_in_zone(bbox, straight_zone):
        return "직진"
    return "미정"

def calculate_angle(dx, dy):
    if dx == 0:
        return 90 if dy > 0 else -90  # 위쪽 이동(90도), 아래쪽 이동(-90도)
    return math.degrees(math.atan2(dy, dx))
# import math

# def calculate_angle(dx, dy):
#     """
#     이동 벡터 (dx, dy)를 이용해 각도를 계산하는 함수.
#     :param dx: X축 이동 거리
#     :param dy: Y축 이동 거리
#     :return: 이동 각도 (degree)
#     """
#     angle = math.degrees(math.atan2(dy, dx))  # atan2는 (-180 ~ 180) 범위 반환
#     return angle



def determine_movement_direction(track_id, current_position):
    """
    객체의 이동 벡터를 계산하고, 실제 이동 방향을 분석하여 반환하는 함수.
    - 수평(왼쪽) 이동 → "직진"
    - 아래쪽 이동 → "우회전"
    - 그 외 → "미정"

    :param track_id: 객체의 고유 추적 ID
    :param current_position: 현재 프레임에서의 객체 중심점 (x, y)
    :return: 이동 방향 ("직진", "우회전", "미정")
    """
    if track_id in object_data:
        prev_position = object_data[track_id]["position"]

        # 이동 벡터 계산
        dx = current_position[0] - prev_position[0]  # X축 이동 거리
        dy = current_position[1] - prev_position[1]  # Y축 이동 거리

        # 이동 방향 각도 계산
       # angle = calculate_angle(dx, dy)

        # ✅ 이동 방향 판별
        if dx < 0:  # 왼쪽 방향 (거의 수평 이동)
            direction = "직진"
        elif dy > 0:  # 아래쪽 이동 → 우회전 가능
            direction = "우회전"
        else:
            direction = "미정"  # 방향을 알 수 없는 경우

        # 객체 데이터 업데이트 (현재 위치를 새로운 기준점으로 저장)
        object_data[track_id] = {"position": current_position, "direction": direction}

        return direction
    else:
        # 처음 감지된 객체 → "미정" 상태로 설정
        object_data[track_id] = {"position": current_position, "direction": "미정"}
        return "미정"

# 이동 거리, 속도, 가속도 계산
import numpy as np


def calculate_motion(object_id, current_position, fps, pixel_to_meter_ratio):
    if object_id in object_data:
        prev_data = object_data[object_id]
        prev_pos = prev_data.get("position",current_position)
        prev_distance = prev_data.get("distance",0.0)
        prev_speed = prev_data.get("speed",0.0)

        # 이전 위치가 현재 위치와 동일하면 거리 계산 제외
        #if np.array_equal(prev_pos, current_position):
        #    print(f"[DEBUG] Object {object_id} did not move.")
        #    return 0.0, prev_speed, 0.0


        # 거리 및 속도 계산
        dx = current_position[0]-prev_pos[0]
        dy = current_position[1]-prev_pos[1]
        distance_pixels = np.linalg.norm([dx,dy])

        distance_meters = prev_distance + (distance_pixels * pixel_to_meter_ratio)
        speed = distance_meters * fps
        acceleration = (speed - prev_speed) * fps

        # 가속도가 비정상적으로 클 경우 제한
        if acceleration > 100:
            print(f"[DEBUG] Object {object_id} - Acceleration 너무 큼: {acceleration:.2f}m/s² → 100으로 제한")
            acceleration = 100
                # 이동이 감지되지 않으면 로그를 남기지 않음

        if distance_meters < 0.01:
            return 0.0, prev_speed, 0.0  # 🔹 작은 이동은 무시

        # 디버깅 정보 출력
        #print(f"[DEBUG] Object {object_id} - Distance: {distance_meters:.2f}m, Speed: {speed:.2f}m/s, Accel: {acceleration:.2f}m/s²")

        # 객체 정보 업데이트
        object_data[object_id] = {"position": current_position, "distance": distance_meters, "speed": speed}
        return distance_meters, speed, acceleration

    else:
        # 객체가 처음 등장했을 때 초기화
        object_data[object_id] = {"position": current_position,"distance":0.0, "speed": 0.0}
        #print(f"[DEBUG] Object {object_id} initialized.")
        return 0.0, 0.0, 0.0


# 객체 클래스 변경 함수
def update_object_class(track_id, new_class):
    track_id = str(track_id)

    if track_id not in object_class_counts:
        object_class_counts[track_id] = Counter()

    # 새로운 클래스를 카운트
    object_class_counts[track_id][new_class] += 1

    # 가장 빈도가 높은 클래스를 가져옴
    most_common_class = object_class_counts[track_id].most_common(1)[0][0]
    #print(f"[DEBUG] Object ID {track_id} - Most Common Class: {most_common_class}")
    class_label = "car" if most_common_class == 2 else "motorcycle"

    # 객체의 클래스가 결정되면 모든 프레임에서 해당 클래스를 업데이트
    for obj in log_data:
        if obj["Object ID"] == track_id:
            obj["Class"] = class_label

    # interval_data에도 반영 (추가된 부분)
    for interval in interval_data:
        if interval["Object 1 ID"] == track_id:
            interval["Object 1 Class"] = class_label
        if interval["Object 2 ID"] == track_id:
            interval["Object 2 Class"] = class_label
    return most_common_class

    # 추가로 객체 정보 갱신
    for track in tracker.tracks:
        if track.track_id == track_id:
            track.class_id = most_common_class

    return most_common_class

# Interval 데이터 생성
def update_interval_data(frame_num, log_data, pixel_to_meter_ratio):
    global interval_data, pair_code_map, pair_code_counter

    right_turn_vehicles = [obj for obj in log_data if obj["Direction"] == "우회전"]
    straight_vehicles = [obj for obj in log_data if obj["Direction"] == "직진" or obj["Direction"]=="미정"]

    for obj1 in right_turn_vehicles:
        for obj2 in straight_vehicles:
            if not obj1.get("Object ID") or not obj2.get("Object ID"):
                continue

            if obj1["Class"] in ["bus","truck"] or obj2["Class"] in ["bus","truck"]:
                continue

            # 중심점 거리 계산
            #center1 = ((obj1["BBox"][0] + obj1["BBox"][2]) / 2, (obj1["BBox"][1] + obj1["BBox"][3]) / 2)
            #center2 = ((obj2["BBox"][0] + obj2["BBox"][2]) / 2, (obj2["BBox"][1] + obj2["BBox"][3]) / 2)
            lag_spacing_pixels = abs(obj1["BBox"][0] - obj2["BBox"][0])
            lag_spacing_meters = lag_spacing_pixels * pixel_to_meter_ratio

            # 고유 Pair Code 생성 및 관리
            pair_key = (obj1["Object ID"], obj2["Object ID"])
            if pair_key not in pair_code_map:
                pair_code_map[pair_key] = pair_code_counter
                pair_code_counter += 1

            interval_data.append({
                "Frame": frame_num,
                "Object 1 ID": obj1["Object ID"],
                "Object 2 ID": obj2["Object ID"],
                "Pair Code": pair_code_map[pair_key],
                "Lag Spacing (m)": lag_spacing_meters,
                "Object 1 Speed (m/s)": obj1["Speed (m/s)"],
                "Object 2 Speed (m/s)": obj2["Speed (m/s)"],
                "Object 1 Class": obj1["Class"],
                "Object 2 Class": obj2["Class"],
                "Density": density_per_frame.get(frame_num,0)
            })

# YOLO 결과를 DeepSORT 입력으로 변환
def yolo_to_deepsort(yolo_results, target_classes):
    detections = []
    classes = []
    for det in yolo_results[0].boxes:
        x1, y1, x2, y2 = map(float, det.xyxy[0].cpu().numpy())
        confidence = float(det.conf.cpu().numpy().item())
        class_id = int(det.cls.cpu().numpy())
        if class_id in target_classes:
            detections.append([(x1, y1, x2, y2), confidence])
            classes.append(class_id)

            # 디버깅 로그 추가: YOLO에서 DeepSORT로 전달되는 바운딩 박스 및 신뢰도 출력
            #print(f"YOLO to DeepSORT - BBox: ({x1}, {y1}, {x2}, {y2}), Confidence: {confidence}, Class ID: {class_id}")

    return detections, classes

# 객체 클래스별 색상 반환 함수
def get_class_color(class_id):
    if class_id == 2:  # car
        return (0, 255, 0)  # 초록 (BGR)
    else:
        return (255, 0, 0)  # 파랑 (BGR)

# 수정된 메인 루프

if __name__ == "__main__":
    target_classes = [2, 3]  # car(2), motorcycle(3)

    video_path = "/content/drive/MyDrive/Capstone/11.15 1800-2000/11.15 1930-2000 - 필터링.mp4"
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
      print(f"Error: Unable to open video file {video_path}")

    fps = int(cap.get(cv2.CAP_PROP_FPS))

    if fps <= 0 or fps > 60:
      #print(f"[DEBUG] FPS 값이 비정상적({fps}), 기본값(30)으로 설정합니다.")
      fps = 30

    #print(f"[DEBUG] FPS: {fps}")

    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    if frame_width == 0 or frame_height == 0:
      print("Error: Invalid video frame dimensions")

    pixel_to_meter_ratio = calculate_pixel_to_meter_ratio(frame_width)

    output_directory = "/content/drive/MyDrive/Capstone/Results/11.15 1800-2000/Log video"
    import os
    output_video_path = os.path.join(output_directory, '11.15 1930-2000_processed_video.avi')
    video_writer = cv2.VideoWriter(output_video_path, cv2.VideoWriter_fourcc(*'MJPG'), fps, (frame_width, frame_height))

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        frame_counter += 1

        # YOLO 객체 검출
        results = model(frame, conf=0.3)

        valid_classes = {2:"car",3:"motorcycle"}
        class_counts = {"car":0,"motorcycle":0}

        # YOLO 결과를 DeepSORT로 변환
        detected_classes.clear()
        all_detections = []

        for r in results:
            for box in r.boxes.data.tolist():
              x1, y1, x2, y2, score, class_id = box
              if score >= 0.5:
                class_name = valid_classes[int(class_id)]
                class_counts[class_name] = class_counts.get(class_name, 0) + 1

                bbox = (int(x1), int(y1), int(x2), int(y2))
                if is_bbox_in_zone(bbox, yolo_to_pixel(interval_zone_yolo, frame_width, frame_height)):
                    all_detections.append([int(x1),int(y1),int(x2),int(y2),float(score)])
                    detected_classes.append(int(class_id))

                if class_id in [2,3,5,7]:
                    density_per_frame[frame_counter] = density_per_frame.get(frame_counter, 0) + 1

        # Ensure detected_classes and tracker.tracks have the same length
        if len(all_detections) != len(detected_classes):
            print(f"Warning: Mismatch between detections ({len(all_detections)}) and detected classes ({len(detected_classes)})")

        # DeepSORTWrapper로 추적
        tracker.update(frame, all_detections)
        if not detected_classes:
            print("Warning: No detected classes were found, skipping classification.")

        current_log_data = []
        for track, detected_class_id in zip(tracker.tracks,detected_classes):
            track_id = track.track_id
            x1, y1, x2, y2 = track.bbox

            # DeepSORT에서 제공하는 track_id를 사용하여 class_id를 가져옴
            #class_id = update_object_class(track_id, 2)  # 기본값 2(car)로 설정

            # 디버깅: 클래스 ID와 색상 확인
            #print(f"Track ID: {track_id}, Updated Class ID: {class_id}")

            updated_class_id = update_object_class(track_id, detected_class_id)

            color = get_class_color(updated_class_id)  # 🔹 color 변수 정의 추가

            cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), color, 2)

            if f"ID {track_id}" not in log_data:
                label = "car" if updated_class_id == 2 else "motorcycle"
                direction = object_directions.get(track_id, "미정")

                cv2.putText(frame, f"{label} ID {track_id} {direction}",
                            (int(x1), int(y1) - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)


            # 바운딩 박스 x좌표가 음수인 경우, 추적 중단
            if x1 < 0 or y1 < 0 or x2 < 0 or y2 < 0:
                continue

            current_position = ((x1 + x2) // 2, (y1 + y2) // 2)
            #print(f"[DEBUG] Frame {frame_counter} - Object {track_id} Previous Position: {object_data.get(track_id, {}).get('position', 'N/A')} -> Current Position: {current_position}")

            #unique_id = get_unique_id(track_id)



            # 이동 거리, 속도, 가속도 계산
            distance, speed, acceleration = calculate_motion(track_id, current_position, fps, pixel_to_meter_ratio)

            # 이동 벡터를 기반으로 실제 이동 방향 결정
            direction = determine_movement_direction(track_id, current_position)

            bbox = (int(x1), int(y1), int(x2), int(y2))

            # 방향 분류
            direction = object_directions.get(track_id, "미정")
            if direction == "미정":
                direction = classify_direction(bbox, frame_width, frame_height)
                object_directions[track_id] = direction

            # 로그 데이터 기록
            bbox = (int(x1), int(y1), int(x2), int(y2))

            #if not any(obj["Object ID"] == track_id for obj in log_data):
            if is_bbox_in_zone(bbox, yolo_to_pixel(interval_zone_yolo, frame_width, frame_height)):
                direction = classify_direction(bbox, frame_width, frame_height)
                updated_class_id = update_object_class(track_id, detected_class_id)

                obj_data = {
                    "Frame": frame_counter,
                    "Object ID": track_id,
                    "Class": "car" if updated_class_id == 2 else "motorcycle" ,
                    "Distance (m)": distance,
                    "Speed (m/s)": speed,
                    "Acceleration (m/s^2)": acceleration,
                    "Direction": direction,
                    "BBox": (x1, y1, x2, y2),
                    "Density" : density_per_frame.get(frame_counter,0)
                }
            log_data.append(obj_data)
            current_log_data.append(obj_data)

            # 화면 표시
            label = "car" if updated_class_id == 2 else "motorcycle"
            color = get_class_color(updated_class_id)
#            cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
            cv2.putText(frame, f"{label} ID {track_id} {direction}", (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5,color, 2)

        # Interval 데이터 업데이트
        update_interval_data(frame_counter, current_log_data, pixel_to_meter_ratio)

        if not video_writer.isOpened():
            print("Error: VideoWriter failed to open.")

        if frame is None:
            print(f"Error: Empty frame at {frame_counter}")




        # 비디오 저장
        video_writer.write(frame)
        # 객체 ID 및 바운딩 박스 정보 로그 출력
        #print(f"Object ID: {unique_id}, BBox: ({x1}, {y1}, {x2}, {y2}), Direction: {direction}")


    # 데이터 저장
    log_data_path = "/content/drive/MyDrive/Capstone/Results/11.15 1800-2000/Log data/1930-2000_log3.csv"
    interval_data_path = "/content/drive/MyDrive/Capstone/Results/11.15 1800-2000/Interval data/1930-2000_interval3.csv"
    pd.DataFrame(log_data).to_csv(log_data_path, index=False)
    pd.DataFrame(interval_data).to_csv(interval_data_path, index=False)


    print(f"Log data saved to: {log_data_path}")
    print(f"Interval data saved to: {interval_data_path}")

    cap.release()
    video_writer.release()

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
Speed: 7.1ms preprocess, 20.1ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)
[DEBUG] Object 112 - Acceleration 너무 큼: 115.12m/s² → 100으로 제한
[DEBUG] Object 116 - Acceleration 너무 큼: 335.05m/s² → 100으로 제한

0: 384x640 5 cars, 11.3ms
Speed: 8.6ms preprocess, 11.3ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)
[DEBUG] Object 112 - Acceleration 너무 큼: 124.87m/s² → 100으로 제한
[DEBUG] Object 116 - Acceleration 너무 큼: 279.21m/s² → 100으로 제한

0: 384x640 5 cars, 18.8ms
Speed: 12.4ms preprocess, 18.8ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)
[DEBUG] Object 116 - Acceleration 너무 큼: 307.13m/s² → 100으로 제한

0: 384x640 5 cars, 12.4ms
Speed: 11.3ms preprocess, 12.4ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)
[DEBUG] Object 116 - Acceleration 너무 큼: 308.40m/s² → 100으로 제한

0: 384x640 5 cars, 16.2ms
Speed: 7.8ms preprocess, 16.2ms inference, 2.1ms postprocess per image at shape (1, 3, 384, 64

In [8]:
###### Extracting Frames by video ######


import cv2
import os

def extract_frames(video_path, output_dir, frame_interval=1):
    """
    특정 경로의 비디오 파일에서 프레임을 추출하여 지정된 디렉토리에 저장하는 함수.

    :param video_path: 비디오 파일 경로 (예: "input/video.mp4")
    :param output_dir: 프레임이 저장될 디렉토리 경로 (예: "output/frames")
    :param frame_interval: 프레임을 저장할 간격 (1이면 모든 프레임 저장, 2이면 2프레임마다 저장)
    """
    # 출력 디렉토리 생성 (없으면 자동 생성)
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # 비디오 캡처 객체 생성
    cap = cv2.VideoCapture(video_path)

    if not cap.isOpened():
        print(f"Error: Cannot open video file {video_path}")
        return

    frame_count = 0
    saved_count = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break  # 더 이상 프레임이 없으면 종료

        # 프레임 간격에 따라 저장 (예: 1프레임씩, 2프레임씩 저장 가능)
        if frame_count % frame_interval == 0:
            frame_filename = os.path.join(output_dir, f"frame_{saved_count:04d}.jpg")
            cv2.imwrite(frame_filename, frame)
            saved_count += 1
            print(f"Saved: {frame_filename}")

        frame_count += 1

    cap.release()
    print(f"\n✅ Total {saved_count} frames saved in {output_dir}")

# 사용 예시
video_path = "/content/drive/MyDrive/Capstone/Results/11.15 1800-2000/Log video/11.15 1930-2000_processed_video.avi"  # 입력 비디오 파일 경로
output_dir = "/content/drive/MyDrive/Capstone/Results/11.15 1800-2000/Frames/11.15 1930-2000"  # 저장할 디렉토리
frame_interval = 1  # 모든 프레임 저장 (2이면 2프레임마다 저장)

extract_frames(video_path, output_dir, frame_interval)

Saved: /content/drive/MyDrive/Capstone/Results/11.15 1800-2000/Frames/11.15 1930-2000/frame_0000.jpg
Saved: /content/drive/MyDrive/Capstone/Results/11.15 1800-2000/Frames/11.15 1930-2000/frame_0001.jpg
Saved: /content/drive/MyDrive/Capstone/Results/11.15 1800-2000/Frames/11.15 1930-2000/frame_0002.jpg
Saved: /content/drive/MyDrive/Capstone/Results/11.15 1800-2000/Frames/11.15 1930-2000/frame_0003.jpg
Saved: /content/drive/MyDrive/Capstone/Results/11.15 1800-2000/Frames/11.15 1930-2000/frame_0004.jpg
Saved: /content/drive/MyDrive/Capstone/Results/11.15 1800-2000/Frames/11.15 1930-2000/frame_0005.jpg
Saved: /content/drive/MyDrive/Capstone/Results/11.15 1800-2000/Frames/11.15 1930-2000/frame_0006.jpg
Saved: /content/drive/MyDrive/Capstone/Results/11.15 1800-2000/Frames/11.15 1930-2000/frame_0007.jpg
Saved: /content/drive/MyDrive/Capstone/Results/11.15 1800-2000/Frames/11.15 1930-2000/frame_0008.jpg
Saved: /content/drive/MyDrive/Capstone/Results/11.15 1800-2000/Frames/11.15 1930-2000/frame